In [10]:
import os
import numpy as np
import torch
import time
import cv2

In [3]:
from totalface.model_zoo.model_common import load_onnx,load_openvino

In [17]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

def preprocessing(aimg,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225],is_onnx=False):
    input_img = normalization(aimg,mean_list,std_list) # aimg is RGB
    if is_onnx:
        input_img = input_img.transpose(1,2,0)
        return input_img
    else:
        input_img = torch.tensor(np.expand_dims(input_img,0).astype(np.float32))
        return input_img

In [4]:
# onnx

In [5]:
path = "efficientNetB0_celebA_224.onnx"

In [6]:
model = load_onnx.Onnx_session(path,input_mean=0.0, input_std=1.0,output_sort=True,onnx_device='cuda')

providers: ['CUDAExecutionProvider']


In [8]:
sample_path = "./sample.jpg"

In [18]:
img = cv2.imread(sample_path)
img = preprocessing(img,is_onnx=True)

In [20]:
out = model(img)

[array([[0.03676867, 0.08534276, 0.00254387, 0.8968301 , 0.15468252,
         0.10697055, 0.05702263, 0.00766271]], dtype=float32)]

In [21]:
# openvino

In [27]:
path = "efficientNetB0_celebA_224.vino"

In [28]:
path = [path.split(".vino")[0]+".xml",path.split(".vino")[0]+".bin"]

In [29]:
model = load_openvino.Openvino(path,not_norm=True,torch_image=True,device='CPU')

In [30]:
img = cv2.imread(sample_path)
img = preprocessing(img)

In [31]:
out = model(img)